In [1]:
import org.opencypher.morpheus.api.MorpheusSession

implicit val morpheus: MorpheusSession = MorpheusSession.local()

Intitializing Scala interpreter ...

Spark Web UI available at http://213cfaf5a00b:4040
SparkContext available as 'sc' (version = 2.4.2, master = local[*], app id = local-1586380378815)
SparkSession available as 'spark'


import org.opencypher.morpheus.api.MorpheusSession
morpheus: org.opencypher.morpheus.api.MorpheusSession = MorpheusSession


###### Define Neo4jContext class

In [2]:
import org.opencypher.okapi.neo4j.io.Neo4jConfig
import org.neo4j.driver.v1.{Driver, Session, StatementResult}

case class Neo4jContext(config: Neo4jConfig) {

    val driver = config.driver()
    val session = driver.session()

    def close(): Unit = {
        session.close()
        driver.close()
    }

    def execute(cypher: String): StatementResult = {
        session.run(cypher)
    }
}

import org.opencypher.okapi.neo4j.io.Neo4jConfig
import org.neo4j.driver.v1.{Driver, Session, StatementResult}
defined class Neo4jContext


###### Create Neo4jConfig objects, one for each neo4j instance

In [3]:
import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig

val NEO4J_PEOPLE_URI = sys.env("NEO4J_PEOPLE_URI")
val NEO4J_PEOPLE_USERNAME = sys.env("NEO4J_PEOPLE_USERNAME")
val NEO4J_PEOPLE_PASSWORD = sys.env("NEO4J_PEOPLE_PASSWORD")

val neo4jPeopleConfig = new Neo4jConfig(
    URI.create(NEO4J_PEOPLE_URI),
    user = NEO4J_PEOPLE_USERNAME, 
    password = Some(NEO4J_PEOPLE_PASSWORD), 
    encrypted = false
)

import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig
NEO4J_PEOPLE_URI: String = bolt://neo4j-people:7687
NEO4J_PEOPLE_USERNAME: String = neo4j
NEO4J_PEOPLE_PASSWORD: String = people
neo4jPeopleConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-people:7687,neo4j,Some(people),false,100000,100000,1000,10)


In [4]:
import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig

val NEO4J_CARS_URI = sys.env("NEO4J_CARS_URI")
val NEO4J_CARS_USERNAME = sys.env("NEO4J_CARS_USERNAME")
val NEO4J_CARS_PASSWORD = sys.env("NEO4J_CARS_PASSWORD")

val neo4jCarsConfig = new Neo4jConfig(
    URI.create(NEO4J_CARS_URI),
    user = NEO4J_CARS_USERNAME, 
    password = Some(NEO4J_CARS_PASSWORD), 
    encrypted = false
)

import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig
NEO4J_CARS_URI: String = bolt://neo4j-cars:7687
NEO4J_CARS_USERNAME: String = neo4j
NEO4J_CARS_PASSWORD: String = cars
neo4jCarsConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-cars:7687,neo4j,Some(cars),false,100000,100000,1000,10)


In [5]:
import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig

val NEO4J_INSURANCE_URI = sys.env("NEO4J_INSURANCE_URI")
val NEO4J_INSURANCE_USERNAME = sys.env("NEO4J_INSURANCE_USERNAME")
val NEO4J_INSURANCE_PASSWORD = sys.env("NEO4J_INSURANCE_PASSWORD")

val neo4jInsuranceConfig = new Neo4jConfig(
    URI.create(NEO4J_INSURANCE_URI),
    user = NEO4J_INSURANCE_USERNAME, 
    password = Some(NEO4J_INSURANCE_PASSWORD), 
    encrypted = false
)

import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig
NEO4J_INSURANCE_URI: String = bolt://neo4j-insurance:7687
NEO4J_INSURANCE_USERNAME: String = neo4j
NEO4J_INSURANCE_PASSWORD: String = insurance
neo4jInsuranceConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-insurance:7687,neo4j,Some(insurance),false,100000,100000,1000,10)


###### Create Neo4jContext objects, one for each Neo4jConfig object

In [6]:
val neo4jPeopleContext = new Neo4jContext(neo4jPeopleConfig)

neo4jPeopleContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-people:7687,neo4j,Some(people),false,100000,100000,1000,10))


In [7]:
val neo4jCarsContext = new Neo4jContext(neo4jCarsConfig)

neo4jCarsContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-cars:7687,neo4j,Some(cars),false,100000,100000,1000,10))


In [8]:
val neo4jInsuranceContext = new Neo4jContext(neo4jInsuranceConfig)

neo4jInsuranceContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-insurance:7687,neo4j,Some(insurance),false,100000,100000,1000,10))


###### Initialize Neo4j graph with sample data

In [9]:
neo4jPeopleContext.execute(
    """|CREATE (:Person {name:'Alice', age:18})-[:drives]->(:Car {plate:'AB123AB'})
       |CREATE (:Person {name:'Bob', age:45})-[:drives]->(:Car {plate:'DE456DE'})""".stripMargin
)

res0: org.neo4j.driver.v1.StatementResult = org.neo4j.driver.internal.InternalStatementResult@4d4389c0


In [10]:
neo4jCarsContext.execute(
    """|CREATE (:Car {plate:'AB123AB', power: 130})
       |CREATE (:Car {plate:'DE456DE', power: 95})""".stripMargin
)

res1: org.neo4j.driver.v1.StatementResult = org.neo4j.driver.internal.InternalStatementResult@6f662505


###### Register Neo4j graphs into morpheus

In [11]:
import org.opencypher.morpheus.api.GraphSources
import org.opencypher.okapi.api.graph.Namespace

import org.opencypher.morpheus.api.GraphSources
import org.opencypher.okapi.api.graph.Namespace


In [12]:
morpheus.registerSource(Namespace("people"), GraphSources.cypher.neo4j(neo4jPeopleContext.config))
val peopleGraph = morpheus.catalog.graph("people.graph")

peopleGraph: org.opencypher.okapi.api.graph.PropertyGraph = ScanGraph(NodePattern(NODE(:Car)), NodePattern(NODE(:Person)), RelationshipPattern(RELATIONSHIP(:drives)))


In [13]:
peopleGraph.cypher("MATCH (n) RETURN n").show

╔══════════════════════════════════════════╗
║ n                                        ║
╠══════════════════════════════════════════╣
║ (:`Car` {`plate`: 'AB123AB'})            ║
║ (:`Car` {`plate`: 'DE456DE'})            ║
║ (:`Person` {`age`: 18, `name`: 'Alice'}) ║
║ (:`Person` {`age`: 45, `name`: 'Bob'})   ║
╚══════════════════════════════════════════╝
(4 rows)


In [14]:
morpheus.registerSource(Namespace("cars"), GraphSources.cypher.neo4j(neo4jCarsContext.config))
val carsGraph = morpheus.catalog.graph("cars.graph")

carsGraph: org.opencypher.okapi.api.graph.PropertyGraph = ScanGraph(NodePattern(NODE(:Car)))


In [15]:
carsGraph.cypher("MATCH (n) RETURN n").show

╔═════════════════════════════════════════════╗
║ n                                           ║
╠═════════════════════════════════════════════╣
║ (:`Car` {`plate`: 'AB123AB', `power`: 130}) ║
║ (:`Car` {`plate`: 'DE456DE', `power`: 95})  ║
╚═════════════════════════════════════════════╝
(2 rows)


In [16]:
morpheus.registerSource(Namespace("insurance"), GraphSources.cypher.neo4j(neo4jInsuranceContext.config))
val insuranceGraph = morpheus.catalog.graph("insurance.graph")

insuranceGraph: org.opencypher.okapi.api.graph.PropertyGraph = EmptyGraph()


In [17]:
insuranceGraph.cypher("MATCH (n) RETURN n").show

╔═══╗
║ n ║
╚═══╝
(no rows)


In [18]:
val integratedGraph = morpheus.cypher(
    """|FROM GRAPH people.graph
       |MATCH (p:Person)-[:drives]->(c1:Car)
       |WHERE p.age < 25
       |FROM GRAPH cars.graph
       |MATCH (c2:Car)
       |WHERE c1.plate = c2.plate AND c2.power > 100
       |CONSTRUCT
       |  ON people.graph, cars.graph
       |  CREATE (p)-[:highRisk]->(c2)
       |RETURN GRAPH
    """.stripMargin
  ).getGraph.get

integratedGraph: org.opencypher.okapi.relational.api.graph.RelationalCypherGraph[org.opencypher.morpheus.impl.table.SparkTable.DataFrameTable] = UnionGraph(graphs=[PrefixedGraph(graph=ScanGraph(NodePattern(NODE(:Car)), NodePattern(NODE(:Person)), RelationshipPattern(RELATIONSHIP(:drives)))),PrefixedGraph(graph=ScanGraph(NodePattern(NODE(:Car)))),ScanGraph(RelationshipPattern(RELATIONSHIP(:highRisk)))])


In [19]:
import org.opencypher.morpheus.api.io.neo4j.sync.Neo4jGraphMerge
import org.opencypher.okapi.neo4j.io.MetaLabelSupport.entireGraphName

// Define the node and relationship keys
  // Note that in this example we assume that names and titles uniquely identify people and products
  val nodeKeys = Map("Person" -> Set("name"), "Car" -> Set("plate"))

  // Write the recommendations back to Neo4j
  Neo4jGraphMerge.merge(entireGraphName, integratedGraph, neo4jInsuranceContext.config, Some(nodeKeys))

import org.opencypher.morpheus.api.io.neo4j.sync.Neo4jGraphMerge
import org.opencypher.okapi.neo4j.io.MetaLabelSupport.entireGraphName
nodeKeys: scala.collection.immutable.Map[String,scala.collection.immutable.Set[String]] = Map(Person -> Set(name), Car -> Set(plate))


In [20]:
  // Proof that the write-back to Neo4j worked, retrieve and print updated Neo4j results
  val updatedNeo4jSource = GraphSources.cypher.neo4j(neo4jInsuranceContext.config)
  morpheus.registerSource(Namespace("updated-neo4j"), updatedNeo4jSource)

updatedNeo4jSource: org.opencypher.morpheus.api.io.neo4j.Neo4jPropertyGraphDataSource = Neo4jPropertyGraphDataSource(Neo4jConfig(bolt://neo4j-insurance:7687,neo4j,Some(insurance),false,100000,100000,1000,10),None,false)


In [21]:
import org.opencypher.okapi.api.graph.QualifiedGraphName
                                       
val socialNetworkWithRanks = morpheus.catalog.graph(QualifiedGraphName(Namespace("updated-neo4j"), entireGraphName))
  socialNetworkWithRanks.cypher(
    """MATCH (person:Person)-[:highRisk]->(c:Car)
      |RETURN person, c""".stripMargin).show

╔══════════════════════════════════════════╤═════════════════════════════════════════════╗
║ person                                   │ c                                           ║
╠══════════════════════════════════════════╪═════════════════════════════════════════════╣
║ (:`Person` {`age`: 18, `name`: 'Alice'}) │ (:`Car` {`plate`: 'AB123AB', `power`: 130}) ║
╚══════════════════════════════════════════╧═════════════════════════════════════════════╝
(1 row)


import org.opencypher.okapi.api.graph.QualifiedGraphName
socialNetworkWithRanks: org.opencypher.okapi.api.graph.PropertyGraph = ScanGraph(NodePattern(NODE(:Car)), NodePattern(NODE(:Person)), RelationshipPattern(RELATIONSHIP(:highRisk)), RelationshipPattern(RELATIONSHIP(:drives)))
